In [3]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph , START , END
from typing import TypedDict

In [4]:
llm = ChatOllama(model = 'llama3')

In [5]:
#state
class BlogState(TypedDict):
    topic :  str
    outline : str
    blog : str
    score : int

In [6]:
def create_outline(state : BlogState) -> BlogState:
    topic = state['topic']

    prompt = f'Generate a detailed outline on topic - {topic}'

    outline = llm.invoke(prompt).content

    state['outline'] = outline

    return state

In [7]:
def create_blog(state : BlogState) -> BlogState:
    topic = state['topic']
    outline = state['outline']
    
    prompt = f'Generate a blog on topic - {topic} using the outline - {outline}'

    blog = llm.invoke(prompt).content

    state['blog'] = blog

    return state

In [12]:
def blog_score(state : BlogState) -> BlogState:
    topic = state['topic']
    outline = state['outline']
    blog = state['blog']

    prompt = f'Generate an integer score out of 10 by evaluating the blog - {blog} on topic - {topic} using outline - {outline}'

    score = llm.invoke(prompt).content

    state['score'] = score

    return state

In [13]:
#graph
graph = StateGraph(BlogState)

#nodes
graph.add_node('create_outline' , create_outline)
graph.add_node('create_blog' , create_blog)
graph.add_node('blog_score' , blog_score)

#edges
graph.add_edge(START , 'create_outline')
graph.add_edge('create_outline' , 'create_blog')
graph.add_edge('create_blog' , 'blog_score')
graph.add_edge('blog_score' , END)

#compile 
workflow = graph.compile()

In [14]:
#execute
initial_state = {'topic' : 'Rising Rape Cases in India'}
final_state = workflow.invoke(initial_state)

In [16]:
#result
print(workflow)
print(final_state['topic'])
print(final_state['outline'])
print(final_state['blog'])
print(final_state['score'])

Rising Rape Cases in India
Here is a detailed outline on the topic "Rising Rape Cases in India":

**I. Introduction**

* Brief overview of the rising trend of rape cases in India
* Statistics and data to highlight the gravity of the issue
* Thesis statement: The rise in rape cases in India is a serious concern that requires immediate attention and collective effort from the government, civil society, and individuals.

**II. Causes of Rising Rape Cases**

* Socio-economic factors:
	+ Poverty and lack of education among perpetrators
	+ Gender inequality and patriarchal attitudes
	+ Lack of economic opportunities for women
* Cultural factors:
	+ Harmful cultural practices and traditions
	+ Normalization of sexual violence in media and entertainment
	+ Lack of gender sensitization and awareness
* Political factors:
	+ Corruption and lack of accountability among law enforcement agencies
	+ Inadequate laws and procedures to deal with rape cases
	+ Political pressure and influence on police a

In [18]:
print(final_state['score'])

Score: 9 out of 10

Strengths:

1. The blog provides a comprehensive overview of the rising trend of rape cases in India, highlighting its causes, impact, and response from the government and civil society.
2. It uses relevant data and statistics to support its arguments, making it a credible source of information.
3. The blog raises important questions about gender inequality, patriarchal attitudes, and harmful cultural practices that contribute to sexual violence.
4. It emphasizes the need for a collective effort to address this issue, involving individuals, civil society, and government.

Weaknesses:

1. The blog could benefit from more specific examples of successful initiatives or programs that have helped prevent sexual violence in India.
2. While it acknowledges the role of corruption and lack of accountability among law enforcement agencies, it does not provide concrete solutions to address these issues.
3. Some readers might find the tone of the blog slightly too formal, which